In [2]:
!pip install langchain langchain-experimental langchain-community langchain-openai openai chromadb pypdf sentence_transformers gradio langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_together import Together
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [5]:


def setup_aws_chatbot(pdf_path, together_api_key):
    # Load the PDF document
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Create embeddings
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma.from_documents(pages, embedding_function)

    # Initialize the LLM
    llm = Together(
        model="meta-llama/Llama-2-70b-chat-hf",
        max_tokens=256,
        temperature=0,
        top_k=1,
        together_api_key=together_api_key
    )

    # Setup the retriever
    retriever = db.as_retriever(similarity_score_threshold=0.9)

    # Define the prompt template
    prompt_template = """Please summarize the following AWS-related document in simple words, less than 100 words.
    CONTEXT: {context}"""
    PROMPT = PromptTemplate(template=f"[INST] {prompt_template} [/INST]", input_variables=["context"])

    # Create the retrieval chain
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        input_key='query',
        return_source_documents=True,
        chain_type_kwargs={"prompt":PROMPT},
        verbose=True
    )
    return chain

def summarize_text(chain):
    # Generate a query that asks for a summary
    query = "Please provide a summary of this document."
    response = chain(query)
    return response['result']

# Example usage
pdf_path = "/content/drylab 1.pdf"  # Replace with the correct path to your PDF document
together_api_key = "5c3aaa83cb623ccf29c3038870387e97d17f64112b00d431f55f6fc19e4d75ab"  # Your Together API key

# Setup the chatbot
aws_chatbot_chain = setup_aws_chatbot(pdf_path, together_api_key)

# Summarize the text
summary = summarize_text(aws_chatbot_chain)
print("Summary:", summary)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.
Summary:   Drylab, a film industry technology company, has released its first newsletter of 2017. The company has raised $2.13 million in funding and has welcomed new investors and mentors. They have also extended their team with two permanent developers and two interns. The company has been touring the US, meeting with potential customers and partners, and has attended the National Association of Broadcasters convention. They have also been invited to the World Wide Developers Conference by Apple. The launch of Drylab 3.0 is delayed, but the company is working hard to get solid feedback from pilot users before its launch at the International Broadcasters Convention in Amsterdam. The company's Annual General Meeting will be held on June 16th. Overall, Drylab is working towards its US launch with Dr ylab 3.0 while keeping momentum in Europe with its existing system.
